In [ ]:
# Count-Min sketch with built-in index search
# Top-1024+ 4*1024 CMS


import sys
import os
import time
from probables import (CountMinSketch)
import pandas as pd

def find(e,Topk):
    try:
        index = [ele for ele,i in Topk].index(e)
        return index
    except:
        index=-99
    return index

start=time.time()
filename='webdocs.dat'
filepath=r"X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs\\"
src_data=os.path.join(filepath,filename)
topk=[]
width=512
depth=4
size=1024
# item_count=10000
cms = CountMinSketch(width=512, depth=4)

if os.path.exists(src_data):
    with open(src_data,'r') as file:
        while True:
            line=file.readline().strip('\n')
            if not line:
                print('EOF')
                break
            else:
                #item_count-=1
                # print("read {}th element: {}".format(item_count,element))
                cms.add(line)
                count=cms.check(line)
                if len(topk)==0:
                    topk.append([line,count])
                else:
                    index=find(line,topk)
                    if index<0:
                        #  element not in topk
                        if len(topk)<size:
                            topk.append([line,count])
                        else:
                            topk[-1][0]=line
                            topk[-1][1]=count
                    else:
                        topk[index][1]=count
                topk=sorted(topk,key = lambda topk:topk[1],reverse=True)
                    
    end=time.time()
    print(topk[:20],len(topk))
    print("Total memory {3} bytes :Top-{0} with size {1} bytes+ CMS with size {2} bytes.".format(len(topk),sys.getsizeof(topk),sys.getsizeof(cms._bins),sys.getsizeof(cms._bins)+sys.getsizeof(topk)))
    print("Execution time:{} seconds.".format(str(end-start)))
else:
    print("file doesn't exist")
#　conver Top into df    
templi=[]
for i in topk:
    templi.append([i[0],i[1]])

df=pd.DataFrame(templi,columns=['ID', 'Count'])
name="CMS_webdocs"+'_'+str(size)+'_'+str(depth)+'_'+str(width)
df.to_csv("X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs\\"+name+".csv",index=False)
df.head(50)


path='X:\\NTU\\ML-sketch\\dataset\\Webpage\\webdocs'
groundtruth='webdocs_ground_truth.csv'
final=name+".csv"

grtruth=pd.read_csv(os.path.join(path,groundtruth),nrows=size)
My_result1=pd.read_csv(os.path.join(path,final))

gli = grtruth.values.tolist()
li1= My_result1.values.tolist()

tp=0
fp=0
find=0
err=[]
error=0
for item in li1:
    for element in gli:
        if item[0]==element[0]:
            # print("{},{} vs. {},{}".format(item[0],item[1],element[0],element[1]))
            find+=1
            if item[1]==element[1]:
                tp+=1
            else:
                fp+=1
                error+=abs(item[1]-element[1])/item[1]
print("Find:{},TP:{},FP:{}".format(find,tp,fp))
#print("ARE:{}".format(error/606770))
